In [17]:
import numpy as np
from scipy.integrate import quad

def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)

def toBar(Pa):
    return (Pa / 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius
Ppad = 1 # Bar

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, name, omega, Pc, Tb, Tc, NCParams, Ant = None, Hf = 0): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.name = name
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant = Ant
        self.Hf = Hf

    def Antoine(self, T): # NIST, T em K, P em Bar
        AntA = self.Ant[0]
        AntB = self.Ant[1]
        AntC = self.Ant[2]
        if T < self.Ant[3] or T > self.Ant[4]:
            print(f"{self.name} : Temperatura {T} é fora da faixa para antoine")
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura, P em Bar
        Ant = self.Ant
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        return T

    def Estado(self, P, T): # Retorna o estado do composto
        if T > self.Tc and P > self.Pc:
            print(f"{self.name} : Fluído supercrítico")
            return "gas"
        elif T > self.Tc or self.Ant == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if P == Ppad:
                if T > self.Tb:
                    return "gas"
                else:
                    return "liquid"
            if  P < self.Antoine(T):
                return "vapour"
            elif P == self.Antoine(T):
                print(f"{self.name} : FODEO")
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))

    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / toPa(self.Pc))
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / toPa(self.Pc))
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3) 
        Z = np.roots([1, alpha, beta, gamma]) # Resolução cúbica
        Z = Z[np.isreal(Z)] 
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self): # Por Riedel p147 V.Ness
        return R * self.Tb * ((1.092 * (np.log(self.Pc) - 1.013)) / (0.930 - (self.Tb / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(Ppad, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(Ppad, T): # estado final
                    case "gas" | "vapour":
                        print("caminho gás - gás")
                        H = quad(self.Cp_ig, Tpad, T)[0] + self.Hres(P, T)
                        print(f"{self.name} : {H} J/mol")
                        return H
                    case "liquid":
                        print("caminho gás - líquido")
                        Tvap = self.TAntoine(Ppad)
                        HGas = quad(self.Cp_ig, Tpad, Tvap)[0] + self.Hres(P, Tvap)
                        Hv = self.Hvap()
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        print(f"{self.name} : {H} J/mol")
                        return H
            case "liquid":
                match self.Estado(Ppad, T): #estado final
                    case "gas" | "vapour":
                        print("caminho líquido - gás")
                        Tvap = self.TAntoine(Ppad)
                        # print(f"tvap = {Tvap}")
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        # print(f"Hl = {Hl}")
                        Hv = self.Hvap()
                        # print(f"Hv = {Hv}")
                        HGas = quad(self.Cp_ig, Tvap, T)[0] + self.Hres(P, T)
                        # print(f"HGas = {HGas}")
                        H = Hl + Hv + HGas
                        print(f"{self.name} : {H} J/mol\n")
                        return H
                    case "liquid":
                        print("caminho líquido - líquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        print(f"{self.name} : {H} J/mol")
                        return H

# Componentes

C2H4O = Comp( # 
        "Oxido de etileno",
        0.201, # omega Perry
        toBar(7.26e6), # Pc Perry
        toK(10.4), # Tb
        469.15, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [5.84696, 2022.83, 62.656, 273.4, 304.9], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -52.630) # Hf J/mol  V. Ness

O2 = Comp( # 
        "Oxigênio",
        0.020, # omega Perry
        toBar(5.02e6), # Pc Perry
        toK(-182.953), # Tb
        154.58, # Tc Perry
        [3.639, # Cp A V.Ness
        0.506e-3, # B
        0, # C
        -0.227e5], # D
        # [3.9523, 340.024, --4.144, 54.36, 154.33], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

N2 = Comp( #
        "Nitrogênio",
        0.037, # omega Perry
        toBar(3.39e6), # Pc Perry
        toK(-195.798), # Tb
        126.2, # Tc Perry
        [3.280, # Cp A V.Ness
        0.593e-3, # B
        0, # C
        0.040e5], # D
        # [3.7362, 264.651, -6.788, 63.14, 126], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

C2H4 = Comp( # 
        "Etileno",
        0.086, # omega Perry
        toBar(5.03e6), # Pc Perry
        toK(-103.8), # Tb
        282.34, # Tc Perry
        [1.424, # Cp A V.Ness
        14.394e-3, # B
        -4.392e-6, # C
        0], # D
        [13.871, 2854.7, -36.23, toK(-169.16), toK(76.52)], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 52.510) # Hf J/mol  V. Ness

CO2 = Comp( #
        "Dioxido de carbono",
        0.224, # omega Perry
        toBar(7.39e6), # Pc Perry
        toK(-78.464), # Tb
        304.21, # Tc Perry
        [5.457, # Cp A V.Ness 
        1.045e-3, # B 
        0, # C 
        -1.157e5], # D 
        # [6.81228, 1301.679, -3.494, 154.26, 195.89]   , # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -393.509) # Hf J/mol V. Ness 
    
CH3CHO = Comp( # 
        "Acetaldeído",
        0.292, # omega Perry
        toBar(5.57e6), # Pc Perry
        toK(20.6), # Tb Haynes
        466, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [3.68639, 822.894, -69.899, 293.4, 377.5], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -166.190) # J/mol V. Ness

H2O = Comp( # 
        "Água",
        0.343, # omega Perry
        toBar(21.94e6), # Pc Perry
        toK(99.974), # Tb Haynes
        647.13, # Tc Perry
        [3.470, # Cp A V.Ness
        1.450e-3, # B
        0, # C
        0.121e5], # D
        [4.6543, 1435.264, -64.848, 255.9, 373], # Antoine A, B, C, D, Tmin, Tmax
        Hf = -285.830) # Líquido J/mol Hf V. Ness


class Corrente: # Corrente : Classe que abriga uma corrente toda
    def __init__(self, name, P, T, C2H4O = 0, O2 = 0, N2 = 0, C2H4 = 0, CO2 = 0, H2O = 0, CH3CHO = 0): # pressão, temperatura e vazões molares kmol / h
        self.name =  name
        self.P = P
        self.T = T
        self.C2H4O = C2H4O
        self.O2 = O2
        self.N2 = N2
        self.C2H4 = C2H4
        self.CO2 = CO2
        self.H2O = H2O
        self.CH3CHO = CH3CHO
    def Hcorr(self):
        H = (self.C2H4O * C2H4O.Hmolar(self.P, self.T) + self.O2 * O2.Hmolar(self.P, self.T) + self.N2 * N2.Hmolar(self.P, self.T) 
             + self.C2H4 * C2H4.Hmolar(self.P, self.T) + self.CO2 * CO2.Hmolar(self.P, self.T) + self.H2O * H2O.Hmolar(self.P, self.T) + self.CH3CHO * CH3CHO.Hmolar(self.P, self.T))        
        print(f"{self.name} : {H} kJ/h")
        return H

In [20]:
# TC-01
C3 = Corrente("Corrente 3, Entrada TC-01", 22, toK(30), 0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
C4a = Corrente("Corrente 4a, Saída TC-01", 20, toK(140), 0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
C5 = Corrente("Corrente 5, Entrada TC-01", 19, toK(240), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)
C5a = Corrente("Corrente 5a, Saída TC-01", 19, toK(150), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)

dH34 = C4a.Hcorr() - C3.Hcorr()
dH5 = C5a.Hcorr() - C5.Hcorr()
while dH34 + dH5 > 1:
    C4a.T -= 0.1
    C5a.T = C4a.T + 10
    dH34 = C4a.Hcorr() - C3.Hcorr()
    dH5 = C5a.Hcorr() - C5.Hcorr()
print(f"C4a.T {C4a.T - 273.15}, Delta C3: {dH34}, Delta C5 {dH5} \n dif: {dH34 + dH5}")


caminho gás - gás
Oxido de etileno : 5775.719285212011 J/mol
caminho gás - gás
Oxigênio : 3413.29368403722 J/mol
caminho gás - gás
Nitrogênio : 3324.5956546110847 J/mol
caminho gás - gás
Etileno : 5410.881911404667 J/mol
caminho gás - gás
Dioxido de carbono : 4417.976543962415 J/mol
caminho líquido - gás
Água : 50034.4236258168 J/mol

caminho gás - gás
Acetaldeído : 5587.427966285497 J/mol
Corrente 4a, Saída TC-01 : 37430129.77360393 kJ/h
caminho gás - gás
Oxido de etileno : -872.9234235912049 J/mol
caminho gás - gás
Oxigênio : 26.409523771136122 J/mol
caminho gás - gás
Nitrogênio : 45.11524820524605 J/mol
caminho gás - gás
Etileno : -289.7113552102725 J/mol
caminho gás - gás
Dioxido de carbono : -237.32312446453346 J/mol
caminho líquido - líquido
Água : 492.7741688483268 J/mol
caminho gás - gás
Acetaldeído : -1167.7261932287815 J/mol
Corrente 3, Entrada TC-01 : -536324.6233683099 kJ/h
caminho gás - gás
Oxido de etileno : 6477.27661168173 J/mol
caminho gás - gás
Oxigênio : 3728.6568094

In [21]:
# F-01
# QF1 = C4b - C4a
C4b = Corrente("Corrente 4b, Saída Forno", 20, toK(200),  0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
QF1 = C4b.Hcorr() - C4a.Hcorr() 
print(f"F-01: {QF1} kJ/h")

caminho gás - gás
Oxido de etileno : 9967.779845363611 J/mol
caminho gás - gás
Oxigênio : 5299.632019883222 J/mol
caminho gás - gás
Nitrogênio : 5119.109793360028 J/mol
caminho gás - gás
Etileno : 8927.741679726452 J/mol
caminho gás - gás
Dioxido de carbono : 7121.105529876768 J/mol
caminho líquido - gás
Água : 52190.15965189304 J/mol

caminho gás - gás
Acetaldeído : 9808.282250699636 J/mol
Corrente 4b, Saída Forno : 59363494.329715274 kJ/h
caminho gás - gás
Oxido de etileno : 5353.894592060836 J/mol
caminho gás - gás
Oxigênio : 3213.7000918180306 J/mol
caminho gás - gás
Nitrogênio : 3133.7693315696306 J/mol
caminho gás - gás
Etileno : 5053.5283530764 J/mol
caminho gás - gás
Dioxido de carbono : 4135.836659638009 J/mol
caminho líquido - gás
Água : 49805.15548307407 J/mol

caminho gás - gás
Acetaldeído : 5162.027075164156 J/mol
Corrente 4a, Saída TC-01 : 35145689.289807566 kJ/h
F-01: 24217805.03990771 kJ/h


In [22]:
# R-01
# Reação lei de hess
R = (C5.C2H4O - C4b.C2H4O) * C2H4O.Hf + (C5.C2H4 - C4b.C2H4) * C2H4.Hf + (C5.CO2 - C4b.CO2) * CO2.Hf + (C5.H2O - C4b.H2O) * H2O.Hf + (C5.CH3CHO - C4b.CH3CHO) * CH3CHO.Hf
print(f"Reação: {R} kJ/h")
H45 = C5.Hcorr() - C4b.Hcorr()
print(f"Delta H, 4b - 5 : {H45} kJ/h")
Hr = R + H45
print(f"R-01: {Hr} kJ/h")

Reação: -110322.7191172161 kJ/h
caminho gás - gás
Oxido de etileno : -172753567.86224312 J/mol
caminho gás - gás
Oxigênio : -87211842.73536545 J/mol
caminho gás - gás
Nitrogênio : -83894660.40236051 J/mol
caminho gás - gás
Etileno : -151889909.47199082 J/mol
caminho gás - gás
Dioxido de carbono : -119182355.35740823 J/mol
caminho líquido - gás
Água : -711947571.0111046 J/mol

caminho gás - gás
Acetaldeído : -170929316.21220347 J/mol
Corrente 5, Entrada TC-01 : -1059810067275.7827 kJ/h
caminho gás - gás
Oxido de etileno : -132260210.50616227 J/mol
caminho gás - gás
Oxigênio : -70319615.54417577 J/mol
caminho gás - gás
Nitrogênio : -67924307.05508389 J/mol
caminho gás - gás
Etileno : -118460179.92206028 J/mol
caminho gás - gás
Dioxido de carbono : -94488334.51675622 J/mol
caminho líquido - gás
Água : -692499393.9468163 J/mol

caminho gás - gás
Acetaldeído : -130143873.08973187 J/mol
Corrente 4b, Saída Forno : -787680745184.3461 kJ/h
Delta H, 4b - 5 : -272129322091.43665 kJ/h
R-01: -27212

In [23]:
# TC-02
# QTC2 = C5b - C5a
C5b = Corrente("Corrente 5a, Saída TC-02", 18, toK(50), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)
QTC2 = C5b.Hcorr() - C5a.Hcorr() 
print(f"TC-02: {QTC2} kJ/h")

caminho gás - gás
Oxido de etileno : -5229537.832305215 J/mol
caminho gás - gás
Oxigênio : -8632630.749907693 J/mol
caminho gás - gás
Nitrogênio : -8697663.912939284 J/mol
caminho gás - gás
Etileno : -9910974.036848415 J/mol
caminho gás - gás
Dioxido de carbono : -8358141.37958789 J/mol
caminho líquido - líquido
Água : -32375849.469177395 J/mol
caminho gás - gás
Acetaldeído : -2183187.765392732 J/mol
Corrente 5a, Saída TC-02 : -86335372822.21703 kJ/h
caminho gás - gás
Oxido de etileno : -80254062.20363198 J/mol
caminho gás - gás
Oxigênio : -46820838.10940689 J/mol
caminho gás - gás
Nitrogênio : -45565262.5375025 J/mol
caminho gás - gás
Etileno : -74676952.51938511 J/mol
caminho gás - gás
Dioxido de carbono : -60897624.2483908 J/mol
caminho líquido - gás
Água : -665920398.0871813 J/mol

caminho gás - gás
Acetaldeído : -77892694.05168007 J/mol
Corrente 5a, Saída TC-01 : -585824771237.6418 kJ/h
TC-02: 499489398415.4248 kJ/h


In [24]:
# Dest-01
# Roult na corrente de fundo
C12 = Corrente("Corrente 12, Entrada", 5.8, CH3CHO.TAntoine(5.8), C2H4O = 225.1780362, H2O = 65.83972599, CH3CHO = 2.748741474)
C15 = Corrente("Corrente 15, Topo", 5.4, C2H4O.TAntoine(5.8), C2H4O = 675.5341086, CH3CHO = 5.447855715)
C16 = Corrente("Corrente 16, Fundo", 5.6, H2O.TAntoine(5.6 * (65.83972599 /(65.83972599 + 0.932789569)) ), H2O = 65.83972599, CH3CHO = 0.932789569)
C15c = Corrente("Corrente 15, Refluxo", 5.4, C2H4O.TAntoine(5.4), C2H4O = 450.3560724, CH3CHO = 3.63190381)

# C15c - C15 = TC-06
TC06 = C15c.Hcorr() - C15.Hcorr() 

# C15 + C16 - C15 - C15c = Q, Q = TC-06 + TC-07
TC07 = C15.Hcorr() + C16.Hcorr() - C15.Hcorr() - C15c.Hcorr() - TC06 

print(f"TC-06: {TC06} kJ/h, TC-07: {TC07} kJ/h")

caminho gás - gás
Oxido de etileno : -19927079.613233533 J/mol
caminho gás - gás
Oxigênio : -13318379.515403662 J/mol
caminho gás - gás
Nitrogênio : -13142637.220766548 J/mol
caminho gás - gás
Etileno : -19612440.245767966 J/mol
caminho gás - gás
Dioxido de carbono : -16422936.950938784 J/mol
caminho líquido - líquido
Água : -44743009.705050014 J/mol
caminho gás - gás
Acetaldeído : -18974017.748333413 J/mol
Corrente 15, Refluxo : -9043193116.369144 kJ/h
caminho gás - gás
Oxido de etileno : -21665705.297394954 J/mol
caminho gás - gás
Oxigênio : -14282944.497754602 J/mol
caminho gás - gás
Nitrogênio : -14083014.056263808 J/mol
caminho gás - gás
Etileno : -21156868.860214297 J/mol
caminho gás - gás
Dioxido de carbono : -17705317.925620582 J/mol
caminho líquido - líquido
Água : -47807672.756793104 J/mol
caminho gás - gás
Acetaldeído : -20720985.906080242 J/mol
Corrente 15, Topo : -14748807856.754871 kJ/h
caminho gás - gás
Oxido de etileno : -21665705.297394954 J/mol
caminho gás - gás
Oxigê